In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

<h3>Load html data from Wikipedia</h3>

In [2]:
req = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(req.text, "lxml")

Find the table and select the first table in case there are many (the required table is the first in the list)

In [3]:
table= soup.find_all('table')[0]

<h3>Insert the data from the html table into a Pandas dataframe</h3> 

In [4]:
postal_codes = []
boroughs = []
neighborhoods = []
postalcodes_neighborhoods = {}

In [5]:
for row in table.find_all("tr")[1:]:  # skipping header row
    cells = row.find_all("td")
    postal_code = cells[0].get_text()
    borough = cells[1].get_text()
    neighborhood = cells[2].get_text().rstrip()
    if (borough != "Not assigned"):
        if (neighborhood == "Not assigned"):
            neighborhood = borough   
        postal_codes.append(postal_code)
        boroughs.append(borough)
        neighborhoods.append(neighborhood)

In [6]:
toronto_df = pd.DataFrame(columns = ['PostalCode','Borough','Neighborhood'])
toronto_df.PostalCode = postal_codes
toronto_df.Borough = boroughs
toronto_df.Neighborhood = neighborhoods
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


<h3>Group Neighborhoods belonging to the same postal code</h3>

For this task, I will use the groupby method of pandas dataframes, and then convert the result (series) into a dataframe resetting the index

In [7]:
toronto_grouped = toronto_df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list)

In [8]:
toronto_grouped_df = toronto_grouped.to_frame().reset_index()
toronto_grouped_df['Neighborhood'].apply(str)
toronto_grouped_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


Convert values of Neighborhood column from arrays to strings with neighborhoods separated by comas

In [9]:
toronto_grouped_df['Neighborhood'] = toronto_grouped_df.Neighborhood.apply(lambda x: ', '.join([str(i) for i in x]))

In [10]:
toronto_grouped_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h3>Print the shape of the final dataframe</h3>

In [11]:
toronto_grouped_df.shape

(103, 3)

<h3>Save to csv for the following parts of the assignment</h3>

In [16]:
toronto_grouped_df.to_csv('toronto_data_acquisition.csv', index= False)